In [1]:
print('abcd')

abcd


In [1]:
import copy
import glob
from prettytable import PrettyTable

def check_noise_margin(wp):
    a=[[float(i) for i in l.split()] for l in open(f'Y_wp{wp}.out','r').readlines()]
    yop={x:y for x,y in a}
    b = [[float(i) for i in l.split()] for l in open(f'deriv_Y_wp{wp}.out','r').readlines()]
    c= copy.deepcopy(a)
    c.sort(key=lambda x: abs(x[0]-x[1]))
    vm=c[0][0]
    nm=[(x,y) for x,y in b if y<=-1]
    vil,vih=nm[0][0],nm[-1][0]
    voh,vol=yop[vil],yop[vih]
    return (wp,vm,vil,vih,vol,voh)
table=[['wp','vm','vil','vih','vol','voh']]
for wp in [37,117,197,277]: table.append(check_noise_margin(wp))
tableT=list(zip(*table))
# print(tableT)
pt=PrettyTable(table[0])
pt.add_rows(table[1:])
print(pt)

+-----+------+------+------+-------------+------------+
|  wp |  vm  | vil  | vih  |     vol     |    voh     |
+-----+------+------+------+-------------+------------+
|  37 | 1.33 | 0.96 | 1.48 | 0.243430731 | 3.13616532 |
| 117 | 1.6  | 1.35 | 1.81 | 0.239540941 | 3.02077689 |
| 197 | 1.73 | 1.52 | 1.97 | 0.197538491 | 3.02188905 |
| 277 | 1.8  | 1.62 | 2.06 | 0.171349188 | 3.03557396 |
+-----+------+------+------+-------------+------------+


In [36]:
# 70% to 30% for slew calculation
vul,vll,vmd=3.3*0.7,3.3*0.3,3.3/2
Awf=[[float(i) for i in l.split()] for l in open(f'tr_A.out','r').readlines()]
arise=[t for i,(t,v) in enumerate(Awf[1:-1]) if Awf[i][1]-vmd < 0 and Awf[i+1][1] -vmd >=0]
afall=[t for i,(t,v) in enumerate(Awf[1:-1]) if Awf[i][1]-vmd > 0 and Awf[i+1][1] -vmd <=0]
a_rise_time=arise[-1]
a_fall_time=afall[0]
print(a_fall_time,a_rise_time)
def calc_delay_slew(wp):
    yop=[[float(i) for i in l.split()] for l in open(f'tr_Y_wp{wp}.out','r').readlines()][1000:]
    y_rise_time=[t for i,(t,v) in enumerate(yop[1:-1]) if yop[i][1]-vmd < 0 and yop[i+1][1] -vmd >=0][0]
    y_fall_time=[t for i,(t,v) in enumerate(yop[1:-1]) if yop[i][1]-vmd > 0 and yop[i+1][1] -vmd <=0][-1]
    rise_delay=y_rise_time-a_fall_time
    fall_delay=y_fall_time-a_rise_time
    check_vll = [(t,v>=vll) for t,v in yop]
    check_vul=[(t,v<=vul) for t,v in yop]
    for i,(t,v) in enumerate(yop):
        if i==0: continue
        if check_vll[i][1] and (not check_vll[i-1][1]) and check_vul[i][1] and check_vul[i-1][1]: rise_start_time=t
        if check_vll[i][1] and check_vll[i-1][1] and (not check_vul[i][1]) and check_vul[i-1][1]: rise_end_time=t
        if check_vll[i][1] and check_vll[i-1][1] and check_vul[i][1] and (not check_vul[i-1][1]): fall_start_time=t
        if (not check_vll[i][1]) and check_vll[i-1][1] and check_vul[i][1] and check_vul[i-1][1]: fall_end_time=t
    rise_slew = rise_end_time - rise_start_time
    fall_slew = fall_end_time - fall_start_time
    return wp,rise_delay,rise_slew,fall_delay,fall_slew

table=[['wp','rise_delay','rise_slew','fall_delay','fall_slew']]
for wp in [37,117,197,277]: table.append(calc_delay_slew(wp))
pt=PrettyTable(table[0])
pt.add_rows(table[1:])
print(pt)


2.8005e-09 4.4005e-09
+-----+------------------------+------------------------+-----------------------+------------------------+
|  wp |       rise_delay       |       rise_slew        |       fall_delay      |       fall_slew        |
+-----+------------------------+------------------------+-----------------------+------------------------+
|  37 | 1.5499999999999998e-10 | 8.000000000000014e-11  |  5.49999999999995e-11 | 5.099999999999993e-11  |
| 117 | 7.899999999999983e-11  | 4.400000000000026e-11  | 7.300000000000006e-11 | 4.7000000000000355e-11 |
| 197 | 5.300000000000013e-11  | 3.699999999999977e-11  | 8.299999999999982e-11 | 4.400000000000026e-11  |
| 277 | 3.8000000000000076e-11 | 3.3000000000000196e-11 | 9.099999999999979e-11 | 4.300000000000078e-11  |
+-----+------------------------+------------------------+-----------------------+------------------------+


In [20]:
3.3/2

1.65